# RAG Tutorial 1 - RAG + Gradio

The goal of this tutorial is to explore a set of techniques for giving LLMs context over private, recent or specific data in order to avoid LLMs hallucinations or them not being able to give a proper answer.

This set of techniques are commonly known as RAG, that stands for Retrieval Augmented Generation.
<br>
<br>

_**Warning**_ <br>
Before diving into the world of RAG, we strongly recommend you, mainly if you are not very familiarized with the field, to read the GLOSSARY.md in this repository. It will give you a basic understanding of fundamental topics regarding Generative AI, Machine Learning and LLMs, crutial in order to better understand RAG.

## Tech Stack

The following libraries and technologies will be used in the development of this tutorial and the applications of RAG within it.

* Langchain - Library for development of LLM applications
* Pinecone - Vector Database
* GPT3.5 - OpenAI Large Language Model
* text-embedding-ada-002 - OpenAI Embedding Model
* Gradio - Library for creating interfaces for ML applications

## What is RAG and how it works?

RAG is a technique for giving LLMs context over private, recent or specific data that the large language model had no previous access to. Its goal is to avoid LLMs hallucinations or them not being able to give a proper answer.

The process of creating an infrastructure for implementing RAG consists on 4 steps
* Getting the data (specific context you want the LLM to know)
* Chunking the data (dividing it into small pieces - chunks)
* Embedding the chunks (transforming the chunks into 'lots of numbers' - vectors of dimension _n_)
* Storing the embeddings in a vector database

When this infrastructure is built, the prompting workflow works as below:

![naiveRAGworkflow](imgs/naiverag.jpg)
As shown, the process has 4 main stages:
* Embedding (From both chunks and User Query)
* Retrieval (Of most relevant chunks of docs)
* Augmentation (New Prompt with Context)
* Generation (LLM being fed the New Prompt)

That workflow is showing the most simple process a RAG application undertakes, in which the user prompt (query) is embedded and then a similairty search is conducted in order to find the chunks that are more related to the prompt made. After that, the most related chunks (top-k chunks) are added to the user prompt and fed into the LLM. The LLM now have (hopefully) not just the user prompt but the necessary context to answer it properly. 

# Example Application - Creating a Chat for an internal Document
Now we will conduct an application of the RAG technique. The goal of this example is to explore the different variables of RAG infrastructure and how they help creating an answer provided by the LLM.

At the end of the day, our main goal when applying RAG is to create a proper answer to the final user.

## Application Context

For this application we will create a custom ChatBot that knows private and specific information from an internal document of our college Insper. The document is the "Manual do Aluno Insper" - a document with a set of rules and informations for students of business and economics at Insper.

When asked about specific information that is on this  document, ChatGPT4, for example, give a basic standard (and wrong or insuficient to say the least) answer. Our chatbot,  due to RAG, is able to have the needed context for answering properly.

## 1) Pre-Configuration - Setting API Keys

In [1]:
from dotenv import load_dotenv

import os

load_dotenv('secrets.env')

True

In [3]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_API_ENV = os.getenv("PINECONE_API_ENV")

"""
API keys are how you can access the Embedding Models, Vector Database and 
other stuff needed for the application.

For this code to work, you must have within the same directory a .env file with the following structure:

OPENAI_API_KEY = 'insert here between the '' your API Key'
PINECONE_API_KEY = 'insert here between the '' your API Key'
PINECONE_API_ENV = 'insert here the Pinecone enironment you are in. ex.: us-east-1'

"""

"\nAPI keys are how you can access the Embedding Models, Vector Database and \nother stuff needed for the application.\n\nFor this code to work, you must have within the same directory a .env file with the following structure:\n\nOPENAI_API_KEY = 'insert here between the '' your API Key'\nPINECONE_API_KEY = 'insert here between the '' your API Key'\nPINECONE_API_ENV = 'insert here the Pinecone enironment you are in. ex.: us-east-1'\n\n"

## 2) Loading Documents

In [4]:
from langchain.document_loaders import PyPDFLoader
"""
PyPDFLoader is a class that langchain gives us to facilitate
the process of loading documents of type PDF.
"""

'\nPyPDFLoader is a class that langchain gives us to facilitate\nthe process of loading doccuments of type PDF.\n'

In [5]:
# PDF "Manual do Aluno Insper"
loader = PyPDFLoader('files/manualdoaluno.pdf')

In [7]:
manual = loader.load()

"""
The method .load() from the instance loader from PyPDFLoader class gives us a
list of Document objects for the full PDF.
"""

'\nThe method .load() from the instance loader from PyPDFLoader class gives us a\nDocument object for the full PDF.\n'

In [9]:
manual # Full list

[Document(page_content=' \n \n \n \n                                                                                        Rua Quatá, 300 – Vila Olímpia 04546 -042 São Paulo SP Brasil  \n55 11 4504 -2400 www.insper.edu.br  \n \n Insper  Instituto  de Ensino  e Pesquisa  \nPortaria MEC nº270, 13/02/2020 – D.O.U. 17/02/2020  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nGraduação | Administração                                                          \ne Ciências Econômicas  \nManual do Aluno  \n2024 \n \n \n \n \n \n \nÁrea responsável:  Secretaria Acadêmica de Graduação  \n                                                                 Data de p ublicação:  janeiro /2024  \n  ', metadata={'source': 'files/manualdoaluno.pdf', 'page': 0}),
 Document(page_content=' \n \n \nInsper  Instituto  de Ensino  e Pesquisa  \nPortaria MEC nº270, 13/02/2020 – D.O.U. 17/02/2020  \n \n2 \n \n \nPublicação: 01/ 2024                                                         

In [8]:
manual[1] # One object within the list

Document(page_content=' \n \n \nInsper  Instituto  de Ensino  e Pesquisa  \nPortaria MEC nº270, 13/02/2020 – D.O.U. 17/02/2020  \n \n2 \n \n \nPublicação: 01/ 2024                                                          Rua Quatá, 300 – Vila Olímpia 04546 -042 São Paulo SP Brasil  \n   55 11 4504 -2400 www.insper.edu.br  \n                                                                             \n \nSUMÁRIO  \n1. BOAS -VINDAS  ................................ ................................ ................................ ................................ ................ 4 \n2. SOBRE OS CURSOS  ................................ ................................ ................................ ................................ ......5 \n2.1. MODELO DE FORMAÇÃO INTEGRADA  ................................ ................................ ............................... 5 \n2.2. APRENDIZADO CENTRADO NO ALUNO  ................................ ................................ ..........

## Chunking Documents

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
"""
RecursiveCharacterTextSplitter is a class that langchain gives us to facilitate
the process of dividing our documents in smaller chunks.
"""

'\nRecursiveCharacterTextSplitter is a class that langchain gives us to facilitate\nthe process of dividing our documents in smaller chunks.\n'

In [89]:
# Creating an instance of RecursiveCharacterTextSplitter class
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2500, # Number os characters in each chunk
    chunk_overlap  = 250, # Number of characters common to subsequent chunks
    length_function = len, # Function to count the number of characters.
    # Obs: We could use a different function if we wanted to define chunk size in terms of word, for example
)
"""
As you can note, chunk_size and chunk_overlap receives just a number of "things".
It does not know if it is characters or word, what the length_function considers 
as a unit (either a character, word, sentence or line) defines what the numbers mean.
"""

'\nAs you can note, chunk_size and chunk_overlap receives just a number of "things".\nIt does not know if it is characters or word, what the length_function considers \nas a unit (either a character, word, sentence or line) defines what the numbers mean.\n'

In [90]:
# Creating chunks
texts = text_splitter.split_documents(manual)
"""
Our instance (object) of class RecursiveCharacterTextSplitter
has a method .split_documents() that will chunk the given document using
the rules defined during the creation of the instance (last cell).
This method will return a list of chunks (Document objects - called 'texts' in this example)
"""

"\nOur instance (object) of class RecursiveCharacterTextSplitter\nhas a method .split_documents() that will chunk the given document using\nthe rules defined during the creation of the instance (last cell).\nThis method will return a list of chunks (Document objects - called 'texts' in this example)\n"

In [91]:
texts # Compare it to the full list 'manuals'
# Those document objects are much smaller ~ Almost half the size

[Document(page_content='Rua Quatá, 300 – Vila Olímpia 04546 -042 São Paulo SP Brasil  \n55 11 4504 -2400 www.insper.edu.br  \n \n Insper  Instituto  de Ensino  e Pesquisa  \nPortaria MEC nº270, 13/02/2020 – D.O.U. 17/02/2020  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nGraduação | Administração                                                          \ne Ciências Econômicas  \nManual do Aluno  \n2024 \n \n \n \n \n \n \nÁrea responsável:  Secretaria Acadêmica de Graduação  \n                                                                 Data de p ublicação:  janeiro /2024', metadata={'source': 'files/manualdoaluno.pdf', 'page': 0}),
 Document(page_content='Insper  Instituto  de Ensino  e Pesquisa  \nPortaria MEC nº270, 13/02/2020 – D.O.U. 17/02/2020  \n \n2 \n \n \nPublicação: 01/ 2024                                                          Rua Quatá, 300 – Vila Olímpia 04546 -042 São Paulo SP Brasil  \n   55 11 4504 -2400 www.insper.edu.br  \n         

In [92]:
print(f"Objects in list 'manual': {len(manual)}\nObjects in list 'texts': {len(texts)}")


Objects in list 'manual': 31
Objects in list 'texts': 56


In [93]:
# Evaluating the number of tokens in each chunk
import tiktoken

encoding = tiktoken.encoding_for_model('gpt-3.5-turbo')

# Example of chunk
text_chunk = texts[2].page_content

# Encodind the chunk in tokens
tokens = encoding.encode(text_chunk)

# Counting number of tokens
num_tokens = len(tokens)

print(f"Number of tokens: {num_tokens}\n")

"""
LLMs have a Context Window (CW) as you know from Glossary. Since this CW is defined in terms of tokens,
it is useful to know how long (in tokens) are your chunks, just to ensure that when creating the Augmented Prompt
you do not retrieve many chunks or include too much messages from Chat History to exceed CW's limit.
Exceeding CW's limit will lead to context being "forgotten" by the LLM, so not considered when answering.
"""

Number of tokens: 462



'\nLLMs have a Context Window (CW) as you know from Glossary. Since this CW is defined in terms of tokens,\nit is useful to know how long (in tokens) are your chunks, just to ensure that when creating the Augmented Prompt\nyou do not retrieve many chunks or include too much messages from Chat History to exceed CW\'s limit.\nExceeding CW\'s limit will lead to context being "forgotten" by the LLM, so not considered when answering.\n'

In [94]:
# Creating a list of all 'page_content' from chunks
manual_contents = [chunk.page_content for chunk in texts]

"""
Each chunk is a Document object.
Those objects have an attribute called page_content that stores (I think you can guess) the page content.
We are creating a list with those contents because if we pass them purely into the Embbeding Model, every character will be embbeded alone.
That means one embeddding for each character instead of one for each chunk.
That would be bad for two reasons:
(i) We wouldn't have the chunk embeddings to conduct similarity search and retrieve relevant chunks.
(ii) Every time you create an embedding you are making an API call for the Embedding Model API, so good luck with the costs.
"""

"\nEach chunk is a Document object.\nThose objects have an attribute called page_content that stores (I think you can guess) the page content.\nWe are creating a list with those contents because if we pass them purely into the Embbeding Model, every character will be embbeded alone.\nThat means one embeddding for each character instead of one for each chunk.\nThat would be bad for two reasons:\n(i) We wouldn't have the chunk embeddings to conduct similarity search and retrieve relevant chunks.\n(ii) Every time you create an embedding you are making an API call for the Embedding Model API, so good luck with the costs.\n"

In [95]:
manual_contents

['Rua Quatá, 300 – Vila Olímpia 04546 -042 São Paulo SP Brasil  \n55 11 4504 -2400 www.insper.edu.br  \n \n Insper  Instituto  de Ensino  e Pesquisa  \nPortaria MEC nº270, 13/02/2020 – D.O.U. 17/02/2020  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nGraduação | Administração                                                          \ne Ciências Econômicas  \nManual do Aluno  \n2024 \n \n \n \n \n \n \nÁrea responsável:  Secretaria Acadêmica de Graduação  \n                                                                 Data de p ublicação:  janeiro /2024',
 'Insper  Instituto  de Ensino  e Pesquisa  \nPortaria MEC nº270, 13/02/2020 – D.O.U. 17/02/2020  \n \n2 \n \n \nPublicação: 01/ 2024                                                          Rua Quatá, 300 – Vila Olímpia 04546 -042 São Paulo SP Brasil  \n   55 11 4504 -2400 www.insper.edu.br  \n                                                                             \n \nSUMÁRIO  \n1. BOAS -VINDAS  ...

In [96]:
print(f"Length manual_contents: {len(manual_contents)}")

Length manual_contents: 56


## Embedding Chunks and Creating vector Database

In [23]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings 
import pinecone 

"""
Pinecone is a class from langchain that facilitates our interactions with pinecone,
the vector database to be used in this application

OpenAIEmbeddings is a class from langchain that facilitates our interactions with
OpenAI Embedding Models, making it easier to use them.

pinecone is a library from pinecone that provides an API to work directly with the vector DB.
"""

'\nPinecone is a class from langchain that facilitates our interactions with pinecone,\nthe vector database to be used in this application\n\nOpenAIEmbeddings is a class from langchain that facilitates our interactions with\nOpenAI Embedding Models, making it easier to use them.\n\npinecone is a library from pinecone that provides an API to work directly with the vector DB.\n'

In [25]:
embeddings_model = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,
    model='text-embedding-ada-002' # Embedding Model -> options: 'text-embedding-3-small', 'text-embedding-3-large'
)
"""
Creating an instance of the class OpenAIEmbeddings to use the 
embeddding model 'text-embedding-ada-002' with our OpenAI API key
"""

"\nCreating an instance of the class OpenAIEmbeddings to use the \nembeddding model 'text-embedding-ada-002' with our OpenAI API key\n"

In [28]:
pinecone_client = pinecone.Pinecone(
                                    api_key=PINECONE_API_KEY,
                                    environment=PINECONE_API_ENV
                                    )


"""
pinecone_client is a client object. Basically an instance of a client class that
allows us to interact with services like pinecone vector database.

We need to give our API key as argument so it can validate
it is us (like a "password" for conventional interfaces)
"""


'\npinecone_client is a client object. Basically an instance of a client class that\nallows us to interact with services like pinecone vector database.\n\nWe need to give our API key as argument so it can validate\nit is us (like a "password" for conventional interfaces)\n'

In [102]:
# CASE 1: Initializing a Vector Database for the first time

# Name of our index (Vector DB)
index_name= 'test-manual' 
# Note that you need to create an index in Pinecone website with this same name

manual_search = Pinecone.from_texts(
    texts=manual_contents,
    embedding=embeddings_model,
    index_name=index_name
)

"""
As you can see, the Index (a name for a Vector Database) is being created 
using Pinecone class from Langchain, through its method .from_texts(). 
'texts' is the content to be embedded and stored (our list of 'page_content' created before - called manual_contents)
'embedding' is the embedding model used to embbed the contents
'index_name' is (I think you can also guess) the name you want your Vector DB to have
"""

"\nAs you can see, the Index (a name for a Vector Database) is being created \nusing Pinecone class from Langchain, through its method .from_texts(). \n'texts' is the content to be embedded and stored (our list of 'page_content' created before - called manual_contents)\n'embedding' is the embedding model used to embbed the contents\n'index_name' is (I think you can also guess) the name you want your Vector DB to have\n"

In [103]:
# Our first Similarity Search
query = 'Quais os trade-offs entre estudar economia e administração?'
retrieved_docs = manual_search.similarity_search(query)

"""
User query is defined, and then passed to the method of our Index .similarity_search()
What this method does fundamentally is embedding the query, comparing it to the whole Vector DB and returning
those embeddings (vectors) that were most similar to the query one.
Note though that we do not get the pure vectors, but the chunks that generated those embeddings, 
so we can use them as context. 
"""

# Print retrieved docs
retrieved_docs

[Document(page_content='encontrado no Portal do Aluno.  \nPara ambos os cursos, os alunos devem completar 80 horas de atividades complementares.  \n2.6. Oportunidades de aprendizagem interativa e hands -on \nAo longo do currículo regular, os alunos terão diversas oportunidades para desenvolver importantes \ncompetências profissionais, por meio de experiências de aprendizagem interativas e hands -on. Já \nno primeiro período, na disciplina de Gestão e Empreendedorismo , os alunos trabalham em grupo  \ncom o objetivo de solucionar um desafio, seja de um determinado público -alvo ou um problema de \numa organização ou da sociedade. No 5º período de Administração, grupos de alunos desenvolvem \numa ideia e estruturam um plano de negócio que consiste em cinco componentes: estratégia, \nmarketing, finanças, operações e comportamento or ganizacional. Finalmente, no 6º período, os \nalunos de Administração participam d o programa “Resolução Eficaz de Problemas” \n(www.insper.edu.br/rep ), e os

In [104]:
# CASE 2: You have already created your Index through the previous example
from langchain.docstore.document import Document # Document class is necessary for creating Doc objects langchain can work with (more on that later)

# Calling an existing Index
manual_search = pinecone_client.Index(index_name)

# This function now serves the purpose of .similarity_search() method in the first example
def perform_search(query, k):
    
    # Generate query embeddings
    query_embedding = embeddings_model.embed_query(query)
    
    # Perform similarity search
    retrieved_matches = manual_search.query(vector=query_embedding, top_k=k, include_metadata=True)

    retrieved_docs = [Document(page_content=match['metadata']['text'], metadata={"source": "database"}) for match in retrieved_matches['matches']]
    return retrieved_docs

"""
If you have already created an Index, using the CASE 1 structure will result in extra and uneeded work.
Just because you will be embedding again all the chunks and going through the whole 
process of creating a Vector DB in Pinecone.
For CASE 2, you can simply call that Index you already created and that is stored in your Pinecone account.

Note though that we are using here pinecone instead of the class Pinecone from langchain.
That basically means they will have different methods. That is why we need a function 
to replace .similarity_search()
"""

'\nIf you have already created an Index, using the CASE 1 structure will result in extra and uneeded work.\nJust because you will be embedding again all the chunks and going through the whole \nprocess of creating a Vector DB in Pinecone.\nFor CASE 2, you can simply call that Index you already created and that is stored in your Pinecone account.\n\nNote though that we are using here pinecone instead of the class Pinecone from langchain.\nThat basically means they will have different methods. That is why we need a function \nto replace .similarity_search()\n'

In [101]:
# In case you want to delete all vectors from your index to reboot the process, run the command below. TAKE CARE!
# manual_search.delete(deleteAll=True)

{}

In [105]:
# Performing the same Similarity Search with CASE 2 structure
query = 'Quais os tipos de estágio no Insper?'
retrieved_docs = perform_search(query, k=4)
retrieved_docs

[Document(page_content='Insper  Instituto  de Ensino  e Pesquisa  \nPortaria MEC nº270, 13/02/2020 – D.O.U. 17/02/2020  \n \n12 \n \n \nPublicação: 01/ 2024                                                          Rua Quatá, 300 – Vila Olímpia 04546 -042 São Paulo SP Brasil  \n   55 11 4504 -2400 www.insper.edu.br  \n                                                                             \n \n▪ Vida estudantil;  \n▪ Atendimento a familiares.  \n \n2.19. Núcleo de Carreiras  \nOferece um conjunto de atividades com o propósito de preparar os alunos  para o ingresso no \nmercado de trabalho. São atividades promovidas ao longo de oito períodos letivos e conduzidas por \nespecialistas em desenvolvimento profissional, por meio de workshops , cursos, palestras, debates e \nsessões de orientação individual. Algumas dessas atividades contabilizam horas de atividades \ncomplementares.  Acesse o site do Carreiras (www.insper.edu.br/carreiras ) para conhecer a \nprogramação de atividades por 

So let's do a quick recap on what we have already done:
* Setup Configuration - Creating .env with API Keys, importing API Keys as variables for our code
* Loading our document - Using PyPDFLoader class to get the full document ready
* Chunking - Dividing the full doc in smaller chunks (texts)
* Embedding - Creating our Vector DB, learning how to call it again after created and how to conduct similarity search

Great! Now let's learn how to use all this infrastructure to enhance LLMs capacity.

## Query Augmentation (Adding most similar Context to User Prompt)

In [63]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

"""
OpenAI is a class from langchain that facilitates our interactions with OpenAI LLMs.
(Note that it is imported from the llms module in langchain library)

load_qa_chain is a function that will help us create a chain from langchain 
specially designed for questions answering
(Note that it is indeed imported from the question_answering module with chains module in langchain library)
"""

'\nOpenAI is a class from langchain that facilitates our interactions with OpenAI LLMs.\n(Note that it is imported from the llms module in langchain library)\n\nload_qa_chain is a function that will help us create a chain from langchain \nspecially designed for questions answering\n(Note that it is indeed imported from the question_answering module with chains module in langchain library)\n'

<b>Quick Note before proceeding:</b>

A 'chain' in Langchain is like a Pipeline that facilitates the whole process of an activity.

So when you are using a chain for question answering, or translation, text summarizations or even the whole process of conversational agents, you are basically getting the benefit of starting a full pipeline from a pre-built structure, in which you define just some arguments through parameter setting.

As you can note by the name, chains are the heart of Langchain, and why this library is so useful.

In [106]:
# Initializing an instance of OpenAI class
llm = OpenAI(temperature=0, api_key=OPENAI_API_KEY)

# Creating a chain using our llm
chain = load_qa_chain(llm, chain_type='stuff')

"""
temperature defines how creative or straight to the point the LLM will be.
A low temperatue makes it very strict, while a high one can lead to crazy outputs.

'stuff' chain_type simply "stuffs" the documents you pass to the chain as conext into the prompt.

You can try different chain_type such as 
* 'map_reduce' chain type maps the question to each document and reduces the answers to a single answer
* 'refine' chain type refines the answer iteratively
* 'map_rerank' chain type maps the question to each document, reranks the answers, and reduces them to a single answer.

Note: Those definitions of each chain_type are from a bot that answers questions about langchain in github.
Always important to give credits to bots in case of a revolution.
"""

'\ntemperature defines how creative or straight to the point the LLM will be.\nA low temperatue makes it very strict, while a high one can lead to crazy outputs.\n\n\'stuff\' chain_type simply "stuffs" the documents you pass to the chain as conext into the prompt.\n\nYou can try different chain_type such as \n* \'map_reduce\' chain type maps the question to each document and reduces the answers to a single answer\n* \'refine\' chain type refines the answer iteratively\n* \'map_rerank\' chain type maps the question to each document, reranks the answers, and reduces them to a single answer.\n\nNote: Those definitions of each chain_type are from a bot that answers questions about langchain in github.\nAlways important to give credits to bots in case of a revolution.\n'

In [107]:
# Answer from LLM to our query
print(chain.run(input_documents=retrieved_docs, question=query))

"""
Do you remember Document Class from CASE2?
We needed to structure retrieved docs as a Document object so 
this chain from langchain can properly work with it
"""

 Os tipos de estágio no Insper são: estágio regular, estágio de férias, estágio interno e estágio realizado no exterior.


'\nDo you remember Document Class from CASE2?\nWe needed to structure retrieved docs as a Document object so \nthis chain from langchain can properly work with it\n'

Nice! It is answering properly. Note how it is being very strict and straight to the point (no kindness here). We will see how to adapt that when building the actual chatbot by changing temperature of the LLM but also by adding more instructions.

# Building a ChatBot

You definetely don't want to tell your business colleague that he or she will need to open VSCode and run a langchain chain in order to get a custom answer. So let's see how to implement an actual chatbot with a graphical user interface and that is also more user friendly in a broader sense.

In [108]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

"""
Those imported classes are used to differentiate messages from Humans and the AI.
SystemMessage is also useful to define some direct instructions for the bot.
"""

'\nThose imported classes are used to differentiate messages from Humans and the AI.\nSystemMessage is also useful to define some direct instructions for the bot.\n'

In [109]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model='gpt-3.5-turbo',
    temperature=0.4
)

"""
ChatOpenAI is a class from langchain that facilitates our interactions 
with OpenAI LLMs for chat applications.

Note that here we are specifing the model we will be using - 'gpt-3.5-turbo'

Note also that we are giving our API Key, everytime we call the API for some 
answer or processing there is a cost. Keep that in mind.
"""

"\nChatOpenAI is a class from langchain that facilitates our interactions \nwith OpenAI LLMs for chat applications.\n\nNote that here we are specifing the model we will be using - 'gpt-3.5-turbo'\n\nNote also that we are giving our API Key, everytime we call the API for some \nanswer or processing there is a cost. Keep that in mind.\n"

In [110]:
# Creating a first set of messages
messages = [
    SystemMessage(content='Você é um chatbot especializado em ajudar alunos de administração e economia com dúvidas e questionamentos acerca do Manual do Aluno.'),
    HumanMessage(content='Olá, eu sou um aluno do Insper. Gostaria de saber mais sobre a faculdade e suas regras.'),
    AIMessage(content='Ótimo! Estou aqui para te ajudar com qualquer pergunta acerca do seu curso de economia ou administração e das regras da sua faculdade.'),
    HumanMessage(content='Quais os princípios fundamentais que devem nortear o dia a dia dos membros da comunidade insper?')
]

In [112]:
# Answer from the ChatBot
answer = chat(messages)
print(answer.content)

"""
Note that we are using the chat() instance we created from ChatOpenAI class previously.
"""

Os princípios fundamentais que devem nortear o dia a dia dos membros da comunidade Insper são:

1. **Ética e Integridade:** Agir com ética, honestidade e transparência em todas as relações e atividades.
   
2. **Excelência Acadêmica:** Buscar a excelência no ensino, na pesquisa e na extensão, promovendo o conhecimento e a inovação.

3. **Respeito e Diversidade:** Respeitar a diversidade de ideias, opiniões e origens, promovendo um ambiente inclusivo e acolhedor.

4. **Responsabilidade Social e Ambiental:** Contribuir para o desenvolvimento sustentável da sociedade e do meio ambiente, por meio de ações responsáveis e engajadas.

5. **Colaboração e Trabalho em Equipe:** Valorizar o trabalho em equipe, a colaboração e a cooperação entre todos os membros da comunidade Insper.

Esses princípios são essenciais para promover um ambiente acadêmico e profissional saudável, ético e de qualidade.


'\nNote that we are using the chat() instance we created from ChatOpenAI class previously.\n'

As you can see, we just got a wrong (or insuficient to say the least) answer. There is a specific page in our "Manual do Aluno" that explicitly states the answer for that question as you can see below:



![insperpageexample](imgs/insperpage.jpg)

But let's keep going and learn how to create a new message and increment chat history. We will soon solve this other question with our Augmented Prompt, made possible by the RAG infrastructure we just built in this tutorial.

In [115]:
# Adding Chat Answer to chat history (List of messages - a simple .append()) 
messages.append(answer)

new_prompt = HumanMessage(
    content='São esses os princípios mesmo chat? Ou você gostaria de uma RAG para responder melhor?'

)

# Ading new prompt to the chat history.
messages.append(new_prompt)

# Getting a New Answer for the New Prompt
answer2 = chat(messages)
print(answer2.content)
"""
Note that chat() instance receives as input the whole chat history, with the last message being
the last prompt made by the user
"""

Peço desculpas pela repetição. Vamos tentar outra abordagem. 

Os princípios fundamentais que devem nortear o dia a dia dos membros da comunidade Insper são:

1. **Ética e Integridade:** Agir com ética, honestidade e transparência em todas as relações e atividades.
   
2. **Excelência Acadêmica:** Buscar a excelência no ensino, na pesquisa e na extensão, promovendo o conhecimento e a inovação.

3. **Respeito e Diversidade:** Respeitar a diversidade de ideias, opiniões e origens, promovendo um ambiente inclusivo e acolhedor.

4. **Responsabilidade Social e Ambiental:** Contribuir para o desenvolvimento sustentável da sociedade e do meio ambiente, por meio de ações responsáveis e engajadas.

5. **Colaboração e Trabalho em Equipe:** Valorizar o trabalho em equipe, a colaboração e a cooperação entre todos os membros da comunidade Insper.

Esses princípios são essenciais para promover um ambiente acadêmico e profissional saudável, ético e de qualidade. Espero que esta resposta tenha sido ma

'\nNote that chat() instance receives as input the whole chat history, with the last message being\nthe last prompt made by the user\n'

Well, the ChatBot not only couldn't answer. Let's solve that.

## Adding relevant context to prompt - Creating Augmented prompt

In [116]:
def augment_prompt(query: str):
    ret_docs = perform_search(query, k=5)
    
    source_knowledge = "\n".join([i.page_content for i in ret_docs])

    augmented_prompt = f"""Usando o contexto fornecido, responda a query abaixo. Se a pergunta 
    não estiver relacionada ao contexto, esqueça o contexto dado e responda como você normalmente 
    responderia, pois você ainda é um assistente disposto a ajudar o aluno Insper de economia
    e administração. Se a pergunta estiver relacionada ao contexto, mas seu conhecimento ou o 
    contexto forem insuficiente para responder a query, apenas diga que não sabe.

    Contextos:
    {source_knowledge}

    Query: {query}"""
    return augmented_prompt

"""
Note that this function does almost exactly what we have done before.

It calls our function perform_search, that retrieves the Document Objects of the most similar to the query chunks.
Source_knowledge is then defined as the sum of all the page_content for each chunk.

Finally, it defines a new "Augmented Prompt" that will later on in this application be fed to the LLM instead
of the user raw query.
"""

'\nNote that this function does almost exactly what we have done before.\n\nIt calls our function perform_search, that retrieves the Document Objects of the most similar to the query chunks.\nSource_knowledge is then defined as the sum of all the page_content for each chunk.\n\nFinally, it defines a new "Augmented Prompt" that will later on in this application be fed to the LLM instead\nof the user raw query.\n'

In [118]:
prompt = HumanMessage(
    content=augment_prompt('Quais são os principios que norteiam o dia a dia do aluno insper? Tente de novo.')
)

# Adding prompt to chat history
messages.append(prompt)

# Getting chat answer
rag_answer = chat(messages)

print(rag_answer.content)

Os princípios que norteiam o dia a dia do aluno Insper são:

1. **Comprometimento:** Manifestado na qualidade dos serviços prestados, na atenção à realização de objetivos e metas estabelecidos, em uma atitude colaborativa voltada para o trabalho em equipe.
  
2. **Confiança Mútua:** Adesão aos compromissos assumidos, honestidade, integridade e sinceridade nas relações são condições que reforçam a confiança mútua, essencial para o trabalho em equipe.
  
3. **Responsabilidade:** Todos são responsáveis pela preservação e segurança do patrimônio humano, material e cultural do Insper, pela boa gestão desse patrimônio e pelo cumprimento de leis e acordos.

4. **Valorização da Diversidade:** Estimular a diversidade fortalece o respeito e a aceitação das diferenças, contribuindo para uma equipe mais eficaz na resolução de problemas e alcance de objetivos.


Bingo! It just got the right answer. 

Keep in mind though that there are several things here impacting the quality of this answer, like:
* <b>Size of the chunk:</b> a really small chunk could divide the document in a way every principle would be a chunk, and perform_search() may not retrieve it in this case.
* <b>Model Used:</b> We are using gpt3.5, but we may use several others as LLMs. Models for embeddings will also alter how chunks are embedded, so similarity search may find different top-k for the same query.
* <b>Value of k:</b> The amount of chunks returned as context defines how much context the LLM gets
* <b>System Message and Initial Chat History:</b> Defines how the LLM will behave and what past conversation it will consider.
* <b>Augmented Prompt Structure:</b> How we define the augmented prompt is like how we prompt ChatGPT for example, different prompts will likely lead to different results.

The lesson here is that you have plenty of room for testing and adapting.

## Creating a Local ChatBot for testing

In [119]:
import gradio as gr

def predict(message, history):
    history_langchain_format = [SystemMessage(content='Você é um assistente que ajuda alunos Insper a entenderem as regras e definições do Manual do Aluno de Administração e Economia Insper, você é muito do bem e gente boa. Responda sempre na língua na qual dor perguntado, pois você também ajuda estudantes internacionais. Busque responder as perguntas para ajudar os alunos Insper quando conseguir!')]
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=augment_prompt(message)))
    gpt_response = chat(history_langchain_format)
    return gpt_response.content

"""
Gradio is the framework for creating interfaces for ML applications that we 
mentioned in the beggining of the tutorial.

We will use gr method .ChatInterface() and .launch()

.ChatInterface() expects a function that receives both message and history as inputs.

message is the user query
history is a list of pairs (UserMessage-BotResponse)

After feeding this function to .ChatInterface() we can simply call .launch() to have a local version 
of our new RAG-powered chatbot.

Note that ou function (predict in this case) is using our function augment_prompt(), that calls several other
functions and structures we previously built here. So our local ChatBot will have the context of our document
'Manual do Aluno' and will work as previously showed in this notebook.
"""

"\nGradio is the framework for creating interfaces for ML applications that we \nmentioned in the beggining of the tutorial.\n\nWe will use gr method .ChatInterface() and .launch()\n\n.ChatInterface() expects a function that receives both message and history as inputs.\n\nmessage is the user query\nhistory is a list of pairs (UserMessage-BotResponse)\n\nAfter feeding this function to .ChatInterface() we can simply call .launch() to have a local version \nof our new RAG-powered chatbot.\n\nNote that ou function (predict in this case) is using our function augment_prompt(), that calls several other\nfunctions and structures we previously built here. So our local ChatBot will have the context of our document\n'Manual do Aluno' and will work as previously showed in this notebook.\n"

In [ ]:
# Finnaly lauching our ChatBot!
gr.ChatInterface(predict).launch(debug=True)

In [ ]:
# You can also use a more beautiful version of Gradio with themes
# Explore more at https://www.gradio.app/guides/theming-guide
# Just be aware it will take longer to run

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.ChatInterface(predict).launch(debug=True)

# Conclusion

Congratulation! You just built your first RAG-Powered ChatBot! Now you can apply that for your personal projects or work problems.

![happycat](imgs/happycatgif.gif)